In [1]:
import json
import time
import psycopg2
import urllib.request
import datetime

from config import config

ecdc_url = "https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/json"

## Exercises 1 and 2

The code in `pipeline.py` creates the database using the configurations from database.ini, it also creates and populates the tables for data source 1 (Covid data) and data source 2 (country data).

We had to create a new CSV file because all the names of countries had trailing whitespace and the names sometimes differed between the JSON and CSV files (e.g. the US is the "United States" in the CSV while it is the "United States Of America" in the JSON, this discrepancy changes the answer of query 3).

We also have to copy the CSV to `C:\\Users\\Public` because of permissions issues associated with the `COPY` command, because of this loading the CSV, and thus filling the second table only works on Windows. In Linux, the file would have to be copied to the `/tmp` folder.

The greatest bottleneck and source of variance in the performance of the code in `pipeline.py` is the request made to the ECDC URL, it can take anywhere between 40 seconds and a several minutes to read, decode and deserialize the JSON file to a Python object. it also possible for the execution to exit with an imcomplete read exception, in this case we have a fallback JSON file in the repository which we use in case this exception is triggered and is updated when the data is succesfully read from the URL.

In [7]:
# only works on Windows
%run pipeline.py

Start Execution: 0.01s
database "covid_db" already exists

Created Database: 0.05s
Retrieved data: 13.73s
Created covid_stats table: 13.79s
Filled covid_stats table: 24.5s
Created country_stats table: 24.54s
Filled country_stats table: 24.68s


## Exercises 3 and 4
The view takes roughly 0.1 seconds to be created and while the other queries take a similar amount of time to execute, meaning the bottleneck of the system is reading the data from the ECDC URL. Still, it would improve performance if we didn't multiply the queries by 100000 or 1000000 since the order would be preserved.

For query 4, the question asks to list the data for all regions, which is a column in the second dataset, for 31/07/2020. The view from exercise 3 was created with only the latest number of cases, so we create a second view in order to answer this query.

In [1]:
%run queries.py

Start execution: 0.0s
Created view: 0.16s
Query 1
What is the country with the highest number of Covid-19 cases per 100 000 inhabitants at 31/07/2020?

Answer 1
The country with the highest case rate per 100000 in 31/07/2020 is Qatar with a case rate of 3856.46 per 100000.

Answered in 0.17s

Query 2
What are the 10 countries with the lowest number of Covid-19 cases per 100 000 inhabitants at 31/07/2020?

Answer 2
The 10 countries with the lowest Covid-19 case rate per 100000 are Laos, Vietnam, Myanmar, United Republic Of Tanzania, Papua New Guinea, Cambodia, Timor Leste, Taiwan, Uganda, Fiji.

Answered in 0.17s

Query 3
What are the top 10 countries with the highest number of cases among the top 20 richest countries (by GDP per capita)? (no date mentioned, assumed most recent)


SELECT "Country", cumulative_count, "GDP ($ per capita)"
FROM recent_cases
WHERE "GDP ($ per capita)" IS NOT NULL
ORDER BY cast("GDP ($ per capita)" as int) DESC, cumulative_count
LIMIT 20;

Answer 3
The 10 co

## Exercise 5
We had the idea of trying to find a URL that holds only the most recent data instead of the whole series, this would likely greatly increase the performance of the pipeline. Alas, we could not find such dataset in the right format and with the correct information (especifically, the 14-day notification rate per 100000 inhabitants).

The second idea was to incorporate vaccination rates per country to the database, this was not done because we ran out of time.